Original code is written by [@Atsuhiko](https://github.com/Atsuhiko)

In [19]:
!nvidia-smi

Wed Nov 13 07:10:15 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.02              Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:01:00.0  On |                  Off |
|  0%   30C    P8             16W /  450W |   20875MiB /  24564MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
# 基本パラメータ
model_id = "Qwen/Qwen2.5-0.5B-Instruct"
peft_name = "NEFTune_Qwen2.5-0.5B-inst_T4_1ep"
output_dir = "output_neftune"


In [21]:
# ライブラリーのインストール
import torch
import wandb
from torch import cuda, bfloat16
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    HfArgumentParser,
    pipeline,
    logging,
)
from datasets import load_dataset
from peft import LoraConfig, PeftModel
from trl import SFTConfig, SFTTrainer

# trl: Transformer Reinfocement Learning, DPOにも対応している

In [22]:
from huggingface_hub import notebook_login

notebook_login()


In [23]:
# 量子化設定
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# モデルの設定
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    # token=token, # HuggingFaceにログインしておけば不要
    quantization_config=bnb_config,  # 量子化
    device_map="auto",
    torch_dtype=torch.bfloat16,
    # attn_implementation="flash_attention_2" # T4では使えないし、RTX4090などの消費者用GPUでも対応していない
)

# tokenizerの設定
tokenizer = AutoTokenizer.from_pretrained(
    model_id, padding_side="right", add_eos_token=True
)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

## ファインチューニング前のモデルでテキスト生成テスト

In [24]:
%%time
messages = [
    {"role": "system", "content": "あなたは日本語で回答するアシスタントです。"},
    {"role": "user", "content": "大阪でおいしいものはなんですか？"},
]

input_ids = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, return_tensors="pt"
).to(model.device)

# terminators = [
#     tokenizer.eos_token_id,
#     tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]
# Qwen用に変更
# Modify the terminators list to only include the eos_token_id
terminators = [
    tokenizer.eos_token_id
]  # Remove tokenizer.convert_tokens_to_ids("<|eot_id|>")

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.8,  # クリエイティブ（再現性なし）
    top_p=0.8,  # 多様な選択肢から単語を選ぶ
    pad_token_id=tokenizer.eos_token_id,  # 追加
    attention_mask=torch.ones(input_ids.shape, dtype=torch.long).cuda(),  # 追加
)
response = outputs[0][input_ids.shape[-1] :]

# print(tokenizer.decode(response, skip_special_tokens=True))

import textwrap

s = tokenizer.decode(response, skip_special_tokens=True)
s_wrap_list = textwrap.wrap(s, 50)  # 50字で改行したリストに変換
print("\n".join(s_wrap_list))


大阪の美味しいお酒は、大阪天守りに由来した「大阪天井酒」が挙げられます。この酒は1948年から販売さ
れ、現在も大阪の観光地として親しまされています。また、大阪府の名物として知られているのが「味噌汁」と
「ラーメン」といった食べ方や特徴があります。
CPU times: user 3.92 s, sys: 337 ms, total: 4.25 s
Wall time: 4.26 s


## データセットの設定

In [25]:
dataset = load_dataset("./Hands-on/01_Instruction_tuning_QLoRA/dataset", split="train")


In [26]:
dataset[5]


{'output': 'わい、思いますねん。 いいえ。\nステイルメイトとは、引き分けた状態のことやねん。どちらがより多くの駒を捕獲したか、または優勢であるかは関係ない、知らんけど。',
 'index': 5,
 'instruction': 'ステイルメイトの時に、私の方が多くの駒を持っていたら、私の勝ちですか？'}

## プロンプトテンプレートの確認

In [27]:
def formatting_func(example):
    messages = [
        {"role": "system", "content": "あなたは日本語で回答するアシスタントです"},
        {"role": "user", "content": example["instruction"]},
        {"role": "assistant", "content": example["output"]},
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False)


def update_dataset(example):
    example["text"] = formatting_func(example)
    for field in ["index", "category", "instruction", "input", "output"]:
        example.pop(field, None)
    return example


dataset = dataset.map(update_dataset)

print(dataset[5]["text"])

<|im_start|>system
あなたは日本語で回答するアシスタントです<|im_end|>
<|im_start|>user
ステイルメイトの時に、私の方が多くの駒を持っていたら、私の勝ちですか？<|im_end|>
<|im_start|>assistant
わい、思いますねん。 いいえ。
ステイルメイトとは、引き分けた状態のことやねん。どちらがより多くの駒を捕獲したか、または優勢であるかは関係ない、知らんけど。<|im_end|>



## LoRAパラメータの設定

In [28]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear4bit(in_features=896, out_features=896, bias=True)
          (k_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (v_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (o_proj): Linear4bit(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear4bit(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
 

In [29]:
# モデルから（4ビット量子化された）線形層の名前を取得する関数
# https://zenn.dev/yumefuku/articles/llm-finetuning-qlora?fbclid=IwY2xjawEih_9leHRuA2FlbQIxMQABHXbPcwqf0DgjPSI9dMMqyuQhUV2z1m2QZLepRWytrm3LOLQkHz9lrETzEg_aem_UTJYtvb55qSBL8Qi3Lttwg
import bitsandbytes as bnb


def find_all_linear_names(model):
    target_class = bnb.nn.Linear4bit
    linear_layer_names = set()
    for name_list, module in model.named_modules():
        if isinstance(module, target_class):
            names = name_list.split(".")
            layer_name = names[-1] if len(names) > 1 else names[0]
            linear_layer_names.add(layer_name)
    if "lm_head" in linear_layer_names:
        linear_layer_names.remove("lm_head")
    return list(linear_layer_names)


# 線形層の名前を取得
target_modules = find_all_linear_names(model)
print(target_modules)


['q_proj', 'o_proj', 'up_proj', 'k_proj', 'gate_proj', 'v_proj', 'down_proj']


In [30]:
peft_config = LoraConfig(
    r=8,
    # lora_alpha=32,
    lora_alpha=16,  # 変更
    lora_dropout=0.05,
    target_modules=target_modules,
    bias="none",
    task_type="CAUSAL_LM",
    modules_to_save=["embed_tokens"],
)

## 学習パラメータの設定

In [31]:
eval_steps = 20
save_steps = 20
logging_steps = 20

training_arguments = SFTConfig(
    bf16=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=16,
    num_train_epochs=3,  # 3エポックに変更
    optim="adamw_torch_fused",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    warmup_steps=100,
    group_by_length=True,
    # report_to="wandb"
    report_to="none",  # 変更
    logging_steps=logging_steps,  # 追加
    eval_steps=eval_steps,  # 追加
    save_steps=save_steps,  # 追加
    output_dir=output_dir,  # 追加
    save_total_limit=3,  # 追加
    push_to_hub=False,
    auto_find_batch_size=True,  # 追加：これを入れないとGPUメモリがオーバーフローする
    packing=True,
    max_seq_length=1024,
    neftune_noise_alpha=5,  # ★★NEFTune設定
    dataset_text_field="text",
)

## SFTrainerの設定、学習の実行

In [32]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_arguments,
)

wandb.init(project="Qwen2.5_sftqlora")


In [33]:
%%time
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()
model.config.use_cache = True

# QLoRAモデルの保存
trainer.model.save_pretrained(peft_name)

Step,Training Loss
20,2.647200
40,2.330700
60,2.087100
80,2.000200
100,1.924600
120,1.886100


CPU times: user 8min 43s, sys: 2min 28s, total: 11min 12s
Wall time: 11min 44s


In [34]:
# 学習したパラメータの比率確認
trainer.model.print_trainable_parameters()

trainable params: 140,533,760 || all params: 634,566,528 || trainable%: 22.1464
